In [17]:
import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [18]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f) 
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [19]:
stopwords = stopwords.words("english")

In [20]:
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [22]:
data = load_data("ushmm_dn.json")["texts"]
print(data[0][0:100])

 My name David Kochalski. I was born in a small town called , and I was born May 5, 1928.  Well, we 


In [23]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[0][0:90])

name bear small town call bear very hard work child father mother small mill flour buckwhe


In [24]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:20])

['name', 'bear', 'small', 'town', 'call', 'bear', 'very', 'hard', 'work', 'child', 'father', 'mother', 'small', 'mill', 'flour', 'buckwheat', 'prosperous', 'comfortable', 'go', 'school']


In [33]:
#BIGRAMS AND TRIGRAMS
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=100)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return ([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

print (data_bigrams_trigrams[0][0:20])

['name', 'bear', 'small', 'town', 'call', 'bear', 'very', 'hard', 'work', 'child', 'father', 'mother', 'small', 'mill', 'flour', 'buckwheat', 'prosperous', 'comfortable', 'go', 'school']


In [34]:
#TF-IDF REMOVAL
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
# print (corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words  = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

In [25]:
#id2word = corpora.Dictionary(data_words)
#
#corpus = []
#for text in data_words:
#    new = id2word.doc2bow(text)
#    corpus.append(new)
#
#print (corpus[0][0:20])
#
#word = id2word[[0][:1][0]]
#print (word)

[(0, 2), (1, 11), (2, 1), (3, 2), (4, 1), (5, 2), (6, 1), (7, 2), (8, 3), (9, 1), (10, 12), (11, 1), (12, 8), (13, 1), (14, 2), (15, 1), (16, 3), (17, 2), (18, 1), (19, 2)]
able


In [38]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=15,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

# Visualization

In [40]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=15)
vis

C:\Users\nawaz\AppData\Local\anaconda3\Lib\site-packages\sklearn\manifold\_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.085440  0.328522       1        1  13.669046
12    -0.222270  0.227852       2        1  13.187363
14    -0.321910  0.107431       3        1  12.700279
2     -0.041872  0.105404       4        1  11.309294
10     0.224920  0.098961       5        1   8.930620
13    -0.122103  0.343370       6        1   8.448154
7      0.293671 -0.298306       7        1   8.196029
9      0.240601  0.281068       8        1   5.112473
5     -0.289335 -0.207582       9        1   4.012268
3     -0.319533 -0.055763      10        1   3.958588
11    -0.104849 -0.165617      11        1   3.766617
8      0.367601  0.015093      12        1   2.562375
0     -0.088269 -0.331449      13        1   1.729602
6      0.219681 -0.134633      14        1   1.594460
4      0.078226 -0.314352      15        1   0.822832, topic_info=           Term         Freq        Total Category  logprob  loglift
337      ghetto  2427.000000  2427.000000  Default  15.0000  15.0000
852         use  1585.000000  1585.000000  Default  14.0000  14.0000
663    remember  1097.000000  1097.000000  Default  13.0000  13.0000
521      mother   680.000000   680.000000  Default  12.0000  12.0000
569        over   672.000000   672.000000  Default  11.0000  11.0000
...         ...          ...          ...      ...      ...      ...
730   situation    44.157834   214.919125  Topic15  -3.5783   3.2177
4695   interest    13.217143    39.079415  Topic15  -4.7846   3.7161
1312       flag    13.830450    44.027235  Topic15  -4.7392   3.6422
1494     letter    25.708284   259.832985  Topic15  -4.1193   2.4869
2001         th    18.018770   187.333282  Topic15  -4.4747   2.4587

[515 rows x 6 columns], token_table=      Topic      Freq                  Term
term                                       
0         1  0.181194                  able
0         2  0.002132                  able
0         3  0.147087                  able
0         5  0.257936                  able
0         8  0.370916                  able
...     ...       ...                   ...
2551     12  0.513217               zionist
3496      2  0.060706  zionist_organization
3496      5  0.151766  zionist_organization
3496      9  0.758829  zionist_organization
2553     15  0.582225                    ½i

[1344 rows x 3 columns], R=15, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 13, 15, 3, 11, 14, 8, 10, 6, 4, 12, 9, 1, 7, 5])